# Fetching Historical Data

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from sklearn.covariance import LedoitWolf
import ipywidgets as widgets

In [2]:
# Fetch SP100 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/S%26P_100'
tables = pd.read_html(url)
df = pd.read_csv("hackathon_sample_v2.csv")
tickers = df.stock_ticker.unique()
tickers = tickers[~pd.isnull(tickers)].tolist()[:100]

# Fetch historical stock data
start_date = '2000-01-01'
end_date = '2023-07-30'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close'].dropna(how='all', axis=1)

In [ ]:
data

# Preprocessing data

In [28]:
# Calculate log returns
returns = data.pct_change().dropna().apply(lambda x: np.log(1 + x))

In [29]:
# Fill NaN values with mean return
returns = returns.fillna(returns.mean())

# Black Litterman

In [30]:
# Variable for the portfolio weights
w = cp.Variable(n)

# Portfolio expected return
expected_return = mu @ w

# Portfolio variance
variance = cp.quad_form(w, Sigma)

# Investor views
P = np.eye(n)
Q = views

# View uncertainty
Omega = view_uncertainty

# Market equilibrium returns
Pi = market_equilibrium_returns

# Black-Litterman model
tau = 0.1
Omega_inv = np.linalg.inv(Omega)
Sigma_inv = np.linalg.inv(Sigma)

# Posterior expected returns
posterior_return = cp.inv_pos(cp.inv_pos(tau * Sigma_inv) + P.T @ Omega_inv @ P) @ (cp.inv_pos(tau * Sigma_inv) @ Pi + P.T @ Omega_inv @ Q)

# Problem constraints
constraints = [
    cp.sum(w) == 1,  # Sum of weights equals 1
    expected_return >= target_return  # Target expected return constraint
]

# Problem objective
objective = cp.Minimize(variance)

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Optimal portfolio weights
optimal_weights = w.value

In [ ]:
# Plot optimal portfolio weights
plt.figure(figsize=(10, 6))
plt.bar(returns.columns, optimal_weights)
plt.xlabel('Asset')
plt.ylabel('Weight')
plt.title('Optimal Portfolio Weights (Black-Litterman Model)')
plt.xticks(rotation=90)

plt.show()